## TF-IDF 알고리즘
- 예제 : https://wikidocs.net/24603

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
path = 'C:/Users/User/Desktop/추천시스템 입문하기/05. 추천시스템 실습하기/input/movies/'

In [3]:
data = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
# overview의 항목 추출 
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
# 전처리 
# overview의 결측치가 있는 항목은 모두 제거 
data = data[data['overview'].notnull()].reset_index(drop=True)
data.shape

(44512, 24)

In [6]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거 
# https://wikidocs.net/22530
tfidf = TfidfVectorizer(stop_words='english')

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(44512, 75827)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 만일 여기서 메모리 에러가 발생하신 분은 TF-IDF의 파라미터를 수정해줘서 다시 돌리면 됩니다. 
# tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# 그래도, 안되는 경우에는 문서의 수를 조금 줄여서 실행해보시길 바랍니다. 
# data = data.loc[0:10000].reset_index(drop=True)

In [8]:
cosine_matrix.shape

(44512, 44512)

In [9]:
np.round(cosine_matrix, 4)

array([[1.    , 0.015 , 0.    , ..., 0.    , 0.0059, 0.    ],
       [0.015 , 1.    , 0.0468, ..., 0.    , 0.022 , 0.0092],
       [0.    , 0.0468, 1.    , ..., 0.    , 0.014 , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 1.    , 0.    , 0.    ],
       [0.0059, 0.022 , 0.014 , ..., 0.    , 1.    , 0.    ],
       [0.    , 0.0092, 0.    , ..., 0.    , 0.    , 1.    ]])

In [10]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
movie2id = {}
for i, c in enumerate(data['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [ ]:
# movie2id

In [ ]:
# id2movie

In [11]:
# Toy Story의 id 추출 
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(15282, 0.5321733978946077),
 (2979, 0.47214559370670484),
 (10271, 0.274962516260823),
 (24316, 0.27322653023092314),
 (23646, 0.23543946958082806),
 (28893, 0.22397858775140161),
 (42572, 0.21761842522811847),
 (37778, 0.2159367770908928),
 (41893, 0.20190977282766223),
 (8303, 0.19868494439439036)]

In [12]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Toy Story 3', 0.5321733978946077),
 ('Toy Story 2', 0.47214559370670484),
 ('The 40 Year Old Virgin', 0.274962516260823),
 ('Small Fry', 0.27322653023092314),
 ("Andy Hardy's Blonde Trouble", 0.23543946958082806),
 ('Hot Splash', 0.22397858775140161),
 ('Andy Kaufman Plays Carnegie Hall', 0.21761842522811847),
 ('Superstar: The Life and Times of Andy Warhol', 0.2159367770908928),
 ('Andy Peters: Exclamation Mark Question Point', 0.20190977282766223),
 ('The Champ', 0.19868494439439036)]

In [25]:
# Toy Story의 id 추출 
idx = id2movie['Harry Potter and the Deathly Hallows: Part 1'] 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(10524, 0.5209150051515135),
 (17341, 0.4282339068315694),
 (11894, 0.37795477801204835),
 (13847, 0.3329922387211944),
 (5657, 0.3158407373047729),
 (7701, 0.29861069679640384),
 (23312, 0.20729855142975492),
 (38360, 0.1613506197949252),
 (25693, 0.15899935824704572),
 (30920, 0.15329794536085917)]

In [26]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Harry Potter and the Goblet of Fire', 0.5209150051515135),
 ('Harry Potter and the Deathly Hallows: Part 2', 0.4282339068315694),
 ('Harry Potter and the Order of the Phoenix', 0.37795477801204835),
 ('Harry Potter and the Half-Blood Prince', 0.3329922387211944),
 ('Harry Potter and the Chamber of Secrets', 0.3158407373047729),
 ('Harry Potter and the Prisoner of Azkaban', 0.29861069679640384),
 ('Luv', 0.20729855142975492),
 ('Bullet to Beijing', 0.1613506197949252),
 ('All Relative', 0.15899935824704572),
 ('Cherry, Harry & Raquel!', 0.15329794536085917)]